In [1]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'hw3'))
	print(os.getcwd())
except:
	pass

/home/renjiezhu/school_workspace/recommender_system/hw3


# Homework 3 - Renjie Zhu - A53266114

In [2]:
import random
import numpy as np

In [3]:
import gzip
from collections import defaultdict

def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

In [4]:
bookCount = defaultdict(int)
book_user = defaultdict(set)
bookSet = set()
totalRead = 0
user_read = defaultdict(set)
data = []

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    user_read[user].add(book)
    book_user[book].add(user)
    bookSet.add(book)
    totalRead += 1
    data.append((user, book))

## 1.



In [5]:
training = data[:190000]
validation = data[190000:]
validation_new = []

In [6]:
for user, book in validation:
    sample = random.sample(bookSet.difference(user_read[user]), 1)[0]
    validation_new.append([user, book, 1])
    validation_new.append([user, sample, 0])
validation = np.array(validation_new)

In [7]:
# baseline model
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

popular = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popular.add(i)
    if count > totalRead * 0.58: break

In [8]:
pred = []
for _, book, _ in validation:
    if book in popular:
        pred.append(1)
    else:
        pred.append(0)


In [9]:
valid_gt = validation[:,2].astype(int)

In [10]:
print(f"Baseline accuracy: {sum(pred==valid_gt)/len(pred)}.")

Baseline accuracy: 0.6535.


## 2.

Better threshold

In [11]:
# better threshold
percentile = list(range(40, 60, 1))
popularSet = []
for p in percentile:
    popular = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        popular.add(i)
        if count > totalRead * (p/100): break
    popularSet.append(popular)

    pred = []
    for _, book, _ in validation:
        if book in popular:
            pred.append(1)
        else:
            pred.append(0)
    
    print(f"Baseline accuracy: {sum(pred==valid_gt)/len(pred)}; threshold: {p} th")

Baseline accuracy: 0.6348; threshold: 40 th
Baseline accuracy: 0.6373; threshold: 41 th
Baseline accuracy: 0.63965; threshold: 42 th
Baseline accuracy: 0.64125; threshold: 43 th
Baseline accuracy: 0.6427; threshold: 44 th
Baseline accuracy: 0.6436; threshold: 45 th
Baseline accuracy: 0.64535; threshold: 46 th
Baseline accuracy: 0.64745; threshold: 47 th
Baseline accuracy: 0.6496; threshold: 48 th
Baseline accuracy: 0.6516; threshold: 49 th
Baseline accuracy: 0.6526; threshold: 50 th
Baseline accuracy: 0.6526; threshold: 51 th
Baseline accuracy: 0.65345; threshold: 52 th
Baseline accuracy: 0.65485; threshold: 53 th
Baseline accuracy: 0.6542; threshold: 54 th
Baseline accuracy: 0.65425; threshold: 55 th
Baseline accuracy: 0.6549; threshold: 56 th
Baseline accuracy: 0.6541; threshold: 57 th
Baseline accuracy: 0.6535; threshold: 58 th
Baseline accuracy: 0.6538; threshold: 59 th


The better threshold is 57th percentage from the above test.

## 3.

Jaccard similarity


In [12]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

def Max_Jaccard(user, book, book_user, user_read):
    similarities = []
    b = book_user[book]
    for read in user_read[user]:
        similarities.append(Jaccard(b, book_user[read]))
    
    return max(similarities)

In [13]:
p=0.5
pred = []
for user, book, _ in validation:
    sim = Max_Jaccard(user, book, book_user, user_read)
    if sim > p:
        pred.append(1)
    else:
        pred.append(0)

print(f"Jaccard accuracy: {sum(pred==valid_gt)/len(pred)}; threshold: {p} th")

Jaccard accuracy: 1.0; threshold: 0.5 th
